# Multilingual RAG Search
Go to following link and run the notebook for original output as locally Wikipedia is not woriking.

https://learn.deeplearning.ai/courses/vector-databases-embeddings-applications/lesson/7/application---multilingual-search

In [13]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import json

def json_print(data):
    print(json.dumps(data, indent=2))

In [17]:
import os

import weaviate
from weaviate.embedded import EmbeddedOptions


client = weaviate.Client(
    embedded_options=EmbeddedOptions(),
    additional_headers={
        "X-OpenAI-Api-Key": os.environ['OPENAI_API_KEY']
    }
)

embedded weaviate is already listening on port 8079


### Total vectors stored in this database

In [4]:
print(json.dumps(client.query.aggregate("Wikipedia").with_meta_count().do(), indent=2))

{
  "errors": [
    {
      "locations": [
        {
          "column": 12,
          "line": 1
        }
      ],
      "message": "Cannot query field \"Wikipedia\" on type \"AggregateObjectsObj\".",
      "path": null
    }
  ]
}


### 3. Perform search over them to find concepts

In [5]:
response = (client.query
            .get("Wikipedia",['text', 'title', 'url', 'views', 'lang'])
            .with_near_text({"concepts": "Vacation spots in california"})
            .with_limit(5)
            .do()
           )

json_print(response)

{
  "errors": [
    {
      "locations": [
        {
          "column": 6,
          "line": 1
        }
      ],
      "message": "Cannot query field \"Wikipedia\" on type \"GetObjectsObj\".",
      "path": null
    }
  ]
}


In [6]:
response = (client.query
            .get("Wikipedia",['text', 'title', 'url', 'views', 'lang'])
            .with_near_text({"concepts": "Vacation spots in california"})
            .with_where({
                "path" : ['lang'],
                "operator" : "Equal",
                "valueString":'en'
            })
            .with_limit(3)
            .do()
           )

json_print(response)

{
  "errors": [
    {
      "locations": [
        {
          "column": 6,
          "line": 1
        }
      ],
      "message": "Cannot query field \"Wikipedia\" on type \"GetObjectsObj\".",
      "path": null
    }
  ]
}


In [7]:
response = (client.query
            .get("Wikipedia",['text', 'title', 'url', 'views', 'lang'])
            .with_near_text({"concepts": "Miejsca na wakacje w Kalifornii"})
            .with_where({
                "path" : ['lang'],
                "operator" : "Equal",
                "valueString":'en'
            })
            .with_limit(3)
            .do()
           )

json_print(response)

{
  "errors": [
    {
      "locations": [
        {
          "column": 6,
          "line": 1
        }
      ],
      "message": "Cannot query field \"Wikipedia\" on type \"GetObjectsObj\".",
      "path": null
    }
  ]
}


In [18]:
response = (client.query
            .get("Wikipedia",['text','title','url','views','lang'])
            .with_near_text({"concepts": "أماكن العطلات في كاليفورنيا"})
            .with_where({
                "path" : ['lang'],
                "operator" : "Equal",
                "valueString":'en'
            })
            .with_limit(3)
            .do()
           )

json_print(response)

{
  "errors": [
    {
      "locations": [
        {
          "column": 6,
          "line": 1
        }
      ],
      "message": "Cannot query field \"Wikipedia\" on type \"GetObjectsObj\".",
      "path": null
    }
  ]
}


## Retrieval Augmented Generation

### Single Prompt

In [19]:
prompt = "Write me a facebook ad about {title} using information inside {text}"
result = (
  client.query
  .get("Wikipedia", ["title","text"])
  .with_generate(single_prompt=prompt)
  .with_near_text({
    "concepts": ["Vacation spots in california"]
  })
  .with_limit(3)
).do()

json_print(result)

{
  "errors": [
    {
      "locations": [
        {
          "column": 6,
          "line": 1
        }
      ],
      "message": "Cannot query field \"Wikipedia\" on type \"GetObjectsObj\".",
      "path": null
    }
  ]
}


### Group Task

In [21]:
generate_prompt = "Summarize what these posts are about in two paragraphs."

result = (
  client.query
  .get("Wikipedia", ["title","text"])
  .with_generate(grouped_task=generate_prompt) # Pass in all objects at once
  .with_near_text({
    "concepts": ["Vacation spots in california"]
  })
  .with_limit(3)
).do()

json_print(result)

{
  "errors": [
    {
      "locations": [
        {
          "column": 6,
          "line": 1
        }
      ],
      "message": "Cannot query field \"Wikipedia\" on type \"GetObjectsObj\".",
      "path": null
    }
  ]
}
